In [1]:
# stdlib
import os
from time import sleep

# syft absolute
import syft as sy
from syft import ActionObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/ionesio/workspace/PySyft


In [6]:
client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

Logged into <nice_botvinick: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [7]:
client.api.services.blob_storage.mount_azure(
    account_name="citestingstorageaccount",
    container_name="citestingcontainer",
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name="helmazurebucket",
)
blob_files = client.api.services.blob_storage.get_files_from_bucket(
    bucket_name="helmazurebucket"
)
document = [f for f in blob_files if "testfile.txt" in f.file_name][0]
x = ActionObject.from_obj(document)
x_ptr = x.send(client)

In [8]:
client.worker.start_workers(n=3)

[ContainerSpawnStatus(worker_name='default-pool-2', worker=syft.service.worker.worker_pool.SyftWorker, error=None),
 ContainerSpawnStatus(worker_name='default-pool-3', worker=syft.service.worker.worker_pool.SyftWorker, error=None),
 ContainerSpawnStatus(worker_name='default-pool-4', worker=syft.service.worker.worker_pool.SyftWorker, error=None)]

In [9]:
client.worker

In [19]:
@sy.syft_function_single_use(document=x_ptr)
def print_remote_document(domain, document):
    print(document.read())
    return document.read()

SyftSuccess: Syft function 'print_remote_documento' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [20]:
client.code.request_code_execution(print_remote_documento)

syft.service.request.request.Request

In [21]:
client.requests[-1].approve(approve_nested=True)

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]:  y


Request approved for domain nice_botvinick


SyftSuccess: Request f6e0ce642c3f4cfea0b0da6ab196849d changes applied

In [30]:
job = client.code.print_remote_document(document=x_ptr, blocking=False)
sleep(5)

In [29]:
job.wait().get()

b'abc\n'